In [ ]:
import pandas as pd

# === Paramètres ===
fichier_source = r"C:\Users\admin\Desktop\ETL_Budget_Automation\ETL_Budget_Automation\data\Template_PMP_Projet_Test_Proximus_KPN.xlsx"
nom_feuille = "Budget Projet"
output_path_cms = r"C:\Users\admin\Desktop\ETL_Budget_Automation\ETL_Budget_Automation\outputs\cms.csv"
output_path_integ = r"C:\Users\admin\Desktop\ETL_Budget_Automation\ETL_Budget_Automation\outputs\integ.csv"

# === Lecture brute du fichier (sans header)
df_full = pd.read_excel(fichier_source, sheet_name=nom_feuille, header=None)

# === Supprimer les 5 premières lignes (décalage vertical) et les 4 premières colonnes (décalage horizontal)
df = df_full.iloc[5:, 4:].reset_index(drop=True)
# Utiliser la première ligne comme entête
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)
df.head(50)  # Afficher les premières lignes du DataFrame après nettoyage


,Process / Phase / Operations,Moynes,Nbre,Cout Unitaire (euro),Cout Moyns Prod (euro),Couts Interface de test (euro),Couts Banc de test(euro),Cout Instrument de test(euro)
0,CMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SMT 1,Pochoir,0,400,0,NaN,NaN,NaN
2,NaN,Caisson DEK,0,1200,0,NaN,NaN,NaN
3,NaN,Feeder et Buses,0,1000,0,NaN,NaN,NaN
4,Vague,Cadre vague,0,800,0,NaN,NaN,NaN
5,NaN,Cadre Mini vague,0,400,0,NaN,NaN,NaN
6,Degrappage,Casette,0,3500,0,NaN,NaN,NaN
7,Retouche,JIG retouche,0,1000,0,NaN,NaN,NaN
8,ICT CIE Mere,Interface de test Off line,1,14000,NaN,14000,NaN,NaN
9,ICT CIE FILLE,Interface de test Off line,1,14000,NaN,14000,NaN,NaN


In [4]:
# === Découpage du DataFrame ===

# Trouver la première ligne (après la ligne 0) qui contient plus de 6 NaN
split_index = None
for idx, row in df.iloc[1:].iterrows():  # on saute la première ligne (idx=1)
    if row.isnull().sum() > 6:
        split_index = idx
        break

if split_index is not None:
    # Découper en deux DataFrames
    df_cms = df.iloc[:split_index].copy()
    df_integ = df.iloc[split_index+1:].copy()  # +1 pour sauter la ligne vide
    # Remettre la même entête pour les deux
    df_cms.columns = df.columns
    df_integ.columns = df.columns


In [5]:
# Supprimer la première ligne de df_cms
df_cms = df_cms.iloc[1:].reset_index(drop=True)

# Supprimer la première, l'avant-dernière et la dernière ligne de df_integ
df_integ = df_integ.iloc[1:-2].reset_index(drop=True)

In [6]:

# On suppose que df_integ est déjà chargé
col_name = df_integ.columns[0]

# On récupère les indices où il y a une valeur non NaN
non_nan_indices = df_integ[df_integ[col_name].notna()].index

# Boucle sur ces indices et remplir entre eux
for i in range(len(non_nan_indices) - 1):
    start = non_nan_indices[i]
    end = non_nan_indices[i + 1]
    df_integ.loc[start+1:end-1, col_name] = df_integ.loc[start, col_name]

# Remplir après la dernière valeur non-NaN s'il en reste
last = non_nan_indices[-1]
df_integ.loc[last+1:, col_name] = df_integ.loc[last, col_name]


In [7]:

# On suppose que df_integ est déjà chargé
col_name = df_cms.columns[0]

# On récupère les indices où il y a une valeur non NaN
non_nan_indices = df_cms[df_cms[col_name].notna()].index

# Boucle sur ces indices et remplir entre eux
for i in range(len(non_nan_indices) - 1):
    start = non_nan_indices[i]
    end = non_nan_indices[i + 1]
    df_cms.loc[start+1:end-1, col_name] = df_cms.loc[start, col_name]

# Remplir après la dernière valeur non-NaN s'il en reste
last = non_nan_indices[-1]
df_cms.loc[last+1:, col_name] = df_cms.loc[last, col_name]


In [11]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

df_cms = df_cms.fillna(0).infer_objects()
df_integ = df_integ.fillna(0).infer_objects()


In [12]:
df_cms.head(50)  # Afficher les 10 premières lignes de df_cms

,Process / Phase / Operations,Moynes,Nbre,Cout Unitaire (euro),Cout Moyns Prod (euro),Couts Interface de test (euro),Couts Banc de test(euro),Cout Instrument de test(euro)
0,SMT 1,Pochoir,0,400,0,0,0,0
1,SMT 1,Caisson DEK,0,1200,0,0,0,0
2,SMT 1,Feeder et Buses,0,1000,0,0,0,0
3,Vague,Cadre vague,0,800,0,0,0,0
4,Vague,Cadre Mini vague,0,400,0,0,0,0
5,Degrappage,Casette,0,3500,0,0,0,0
6,Retouche,JIG retouche,0,1000,0,0,0,0
7,ICT CIE Mere,Interface de test Off line,1,14000,0,14000,0,0
8,ICT CIE FILLE,Interface de test Off line,1,14000,0,14000,0,0
9,BP3,Interfaces de test,0,350,0,0,0,0


In [13]:
df_integ.head(50)  # Afficher les 10 premières lignes de df_integ

,Process / Phase / Operations,Moynes,Nbre,Cout Unitaire (euro),Cout Moyns Prod (euro),Couts Interface de test (euro),Couts Banc de test(euro),Cout Instrument de test(euro)
0,MAIN BOARD: Ass CMS-2 et integ,Gabarit de press capots blindage Main,0,700,0,0,0,0
1,MAIN BOARD: Ass CMS-2 et integ,Gabarits d'assemblage interboard,0,800,0,0,0,0
2,RF BOARD: Ass CMS-2 et integ,Montage 2xPads + capots blindages,0,700,0,0,0,0
3,RF BOARD: Ass CMS-2 et integ,Gabarit Press Capots blindages,0,700,0,0,0,0
4,RF BOARD: Ass CMS-2 et integ,Gabarit d'ASS CIE + dissipateur central,0,1000,0,0,0,0
5,RF BOARD: Ass CMS-2 et integ,Gabarit de Mariage ensemble CIE Mère + fille ...,0,1000,0,0,0,0
6,RF BOARD: Ass CMS-2 et integ,Gabarit de vissage support antenne (2xvis),0,1000,0,0,0,0
7,RF BOARD: Ass CMS-2 et integ,Gabarit de plug antennes coax,0,1500,0,0,0,0
8,RF BOARD: Ass CMS-2 et integ,Plateaux thermformé moule proto,0,700,0,0,0,0
9,RF BOARD: Ass CMS-2 et integ,Plateaux thermformé moule série,0,3500,0,0,0,0


In [14]:
# Sauvegarder
df_cms.to_csv(output_path_cms, index=False)
df_integ.to_csv(output_path_integ, index=False)
print(f"✅ Découpage effectué. Fichiers créés :\n- {output_path_cms}\n- {output_path_integ}")

✅ Découpage effectué. Fichiers créés :
- C:\Users\imen\Desktop\ETL_Budget_Automation\outputs\cms.csv
- C:\Users\imen\Desktop\ETL_Budget_Automation\outputs\integ.csv
